In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
from datetime import datetime
#import sqlalchemy as sa

In [2]:
sys.path.append(r"../../src/collectcube")
from db import *

In [4]:
#out_dir = '../../data/LUCinLA_paraguay/smSamp'
#local_db_path = os.path.join(out_dir, 'landcover.db')
out_dir = 'C:/GISprojects/ParaguayValidation/smDistricts'
local_db_path = 'C:/GISprojects/ParaguayValidation/smDistricts/landcover.db'

In [4]:
table_check = check_table(local_db_path,'pixels')

             PID  Center     cent_X     cent_Y   cent_lat  cent_long  ransamp  \
49495  0005499_4       0  2974500.0 -3213020.0 -25.431507 -57.444894        1   
49496  0005499_5       0  2974520.0 -3213020.0 -25.431507 -57.444675        1   
49497  0005499_6       0  2974500.0 -3213030.0 -25.431589 -57.444883        1   
49498  0005499_7       0  2974510.0 -3213030.0 -25.431589 -57.444774        1   
49499  0005499_8       0  2974520.0 -3213030.0 -25.431589 -57.444665        1   

       checked  PID0  PID1          sampgroup  
49495        0  5499     4  D1119_Central_JAS  
49496        0  5499     5  D1119_Central_JAS  
49497        0  5499     6  D1119_Central_JAS  
49498        0  5499     7  D1119_Central_JAS  
49499        0  5499     8  D1119_Central_JAS  


## Data cleaning (e.g. if pandas import below doesn't work)
Note: need for this should be eliminated as more formatting restrictions are placed in db entry functions

### manipulating table (should not need to do this here)

In [16]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
## adding a new column to existing table
c.execute("ALTER TABLE PixelVerification ADD COLUMN entry_lev INTEGER")
## filling column in existing table
#c.execute("UPDATE PixelVerification SET entry_lev = 4 WHERE entry_lev IS NULL")
con.commit()      
c.close()

(1994, '2701_0', 2701, 0, '2021_03-07', 40, 48, 5, '', '', '', '', '', '', '', '', '', 40, '', 60, '', '', '', '', '', '--', '--', '', 4)


### querying table (e.g. for bad records)

In [17]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("SELECT * FROM PixelVerification  WHERE imgDate = '2021_03-07'")
con.commit()
print(c.fetchone()) 

(1994, '2701_0', 2701, 0, '2021_03-07', 40, 48, 5, '', '', '', '', '', '', '', '', '', 40, '', 60, '', '', '', '', '', '--', '--', '', 4)


### fixing bad records

In [19]:
'''
con = sqlite3.connect(local_db_path)
c = con.cursor()
c.execute("UPDATE PixelVerification SET imgDate = '2021-03-07' WHERE imgDate ='2021_03-07'")
con.commit()
c.close()

## Open db table as pandas dataframe

In [20]:
## This uses sqlalchemy but is prefered because it will make sure that formatting is correct
engine = create_engine('sqlite:///'+ local_db_path)
with engine.connect() as con:
    df = pd.read_sql_table('PixelVerification', con)
df.head()

,recID,PID,PID0,PID1,imgDate,LC5,LC,HOMONBHD9,ForestProx,WaterProx,...,MEDVEG,TREEPLANT0,HIGHVEG,TREEPLANT,FOREST,Age,Stability,State,Notes,entry_lev
0,1,0_0,0,0,2000-01-01,0,0,8.0,,,...,,,,,,,--,--,,4
1,2,2500_0,2500,0,2007-06-07,20,31,8.0,,,...,,,,,,,--,--,,4
2,3,2500_0,2500,0,2008-12-03,20,31,8.0,,,...,,,,,,,--,--,,4
3,4,2500_0,2500,0,2013-07-10,20,31,8.0,,,...,,,,,,,--,--,,4
4,5,2500_0,2500,0,2016-05-17,20,31,8.0,,,...,,,,,,,--,--,,4


In [5]:

## This uses only sqllite and can be used to examine table if the above fails
conn = sqlite3.connect(local_db_path, isolation_level=None,
    detect_types=sqlite3.PARSE_COLNAMES)
db_df = pd.read_sql_query("SELECT * FROM PixelVerification", conn)
db_df.head()
db_df.to_csv(os.path.join('C:/GISprojects/ParaguayValidation/smDistricts','sm_test_db.csv'), index=False)